In [1]:
import time
import pandas as pd
import numpy as np
import PySimpleGUI as sg
import datetime
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import pyodbc
import os
import re
import pandas as pd
from io import StringIO
from pathlib import Path
import io 
# from Viator_AllLinks import main as main_alllinks
# from Viator_GetOperator import main as main_getoperator

In [2]:
"""
Summary:
The main function orchestrates the process of reading an Excel file, excluding sheets named 'DONE', 
processing data to remove duplicate 'Tytul URL' entries, and either appending to or overwriting an 
output Excel file based on date comparison. It leverages the functions defined below to handle each step.
"""

# Function to read Excel, ignoring sheets named 'DONE'
def read_excel_sheets(excel_path):
    """Read all sheets from the Excel file except those named 'DONE'."""
    all_sheets_data = pd.read_excel(excel_path, sheet_name=None)
    sheets_data = {sheet_name: data for sheet_name, data in all_sheets_data.items() if sheet_name.upper() != 'DONE'}
    return sheets_data

# Function to process and combine sheets data
def process_sheets(sheets_data):
    """Combine data from all sheets into a single DataFrame and remove duplicates based on 'Tytul URL'."""
    combined_data = pd.DataFrame()
    for sheet, data in sheets_data.items():
        data['Tytul URL'] = data['Tytul URL'].str.lower()
        data.drop_duplicates(subset=['Tytul URL'], inplace=True)
        combined_data = pd.concat([combined_data, data], ignore_index=True)
    return combined_data

# Function to append or overwrite the output file
def update_output_file(output_path, combined_data, input_date):
    """Append to or overwrite the output file based on the 'Data zestawienia' date comparison."""
    if output_path.exists():
        output_data = pd.read_excel(output_path)
        if not output_data.empty and 'Data zestawienia' in output_data.columns and output_data['Data zestawienia'].eq(input_date).any():
            combined_data = pd.concat([combined_data, output_data], ignore_index=True)

        else:
            print("Dates do not match, overwriting the output file.")
    combined_data.to_excel(output_path, index=False, sheet_name='AllLinks')

# Main workflow
def main(input_excel, output_excel):
    """The main function coordinating the reading, processing, and output of Excel data."""
    input_path = Path(input_excel)
    output_path = Path(output_excel)
    
    # Reading the input file
    sheets_data = read_excel_sheets(input_path)

    first_sheet_name = next(iter(sheets_data))
    input_date = sheets_data[first_sheet_name]['Data zestawienia'].iloc[0]

    # Processing sheets data
    combined_data = process_sheets(sheets_data)
    
    # Updating the output file
    update_output_file(output_path, combined_data, input_date)

   
# Today's date to be used in the file name
today_date = datetime.datetime.now().strftime('%Y-%m-%d')

# File paths, replace the placeholders with actual paths

file_output_g = fr"G:\.shortcut-targets-by-id\1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2\MyOTAs\Baza Excel\Get Your Guide\All Links\All Links GYG - {today_date}.xlsx"
file_input_g = fr"G:\.shortcut-targets-by-id\1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2\MyOTAs\Baza Excel\Get Your Guide\GYG - {today_date}.xlsx"
file_input_v = fr"G:\.shortcut-targets-by-id\1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2\MyOTAs\Baza Excel\Viator\Daily\Viator - {today_date}.xlsx"
file_output_v = fr"G:\.shortcut-targets-by-id\1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2\MyOTAs\Baza Excel\Viator\All Links\All Links Viator - {today_date}.xlsx"



# main(file_input_g, file_output_g)
# main(file_input_v, file_output_v)


In [3]:
# df_main = pd.read_excel(r'G:\.shortcut-targets-by-id\1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2\MyOTAs\Pliki firmowe\Operators_GYG.xlsx')
# df_day = pd.read_excel(r'G:\.shortcut-targets-by-id\1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2\MyOTAs\Baza Excel\Viator\All Links\All Links Viator - 2023-03-08.xlsx', sheet_name='AllLinks', header=None)
# df_day =  pd.read_excel(r'G:\.shortcut-targets-by-id\1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2\MyOTAs\Baza Excel\Get Your Guide\AllLinks\All Links GYG - 2023-03-07.xlsx',  sheet_name='AllLinks')
# df_main = pd.read_csv(r'G:\.shortcut-targets-by-id\1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2\MyOTAs\Pliki firmowe\Operators_Groups.csv')

In [4]:
# df_day = df_day[[0, 1, 3, 6, 9]]
# df_day.drop_duplicates('Tytul Url')
# df_day.rename(columns = {0:'Tytul', 1:'Tytul Url', 3:'IloscOpini', 6:'Data zestawienia', 9:'Miasto'})
# df_day.dtypes
# df_main.drop(columns='Tytul', inplace=True)
# df_main


In [5]:
def insert_new_links(path_main, path_daily, sheetExcel):
    get_time = time.time()
    df_main = pd.read_csv(path_main)
    display(df_main)
    #Based on input read files
    if 'Viator' in path_daily:
#         df_day = df_day[[0, 1, 3, 6, 9]]
        try:
            df_day = pd.read_excel(path_daily, sheetExcel)
        except:
            raise Exception("File not found")
#         df_day.rename(columns = {0:'Tytul', 1:'Tytul Url', 3:'IloscOpini', 6:'Data zestawienia', 9:'Miasto'}, inplace=True)
    elif 'GYG' in path_daily:
        try:
            df_day = pd.read_excel(path_daily, sheetExcel)
        except:
            raise Exception("File not found")
    #SHORTEN DFS
#     df_day = df_day.head(100) #short version for testing    
#     display(df_day)
#     df_main = df_main.head(50) #short version for testing
    #____
    
    df_day = df_day[['Tytul', 'Tytul URL', 'IloscOpini' ,'Data zestawienia', 'Miasto']]    
    df_day.rename(columns={'Tytul URL': 'Link', 'IloscOpini': 'Reviews', 'Miasto': 'City', 'Data zestawienia': 'Date input'}, inplace=True)
    df_day.insert(len(df_day.columns), 'Date update', df_day['Date input'])
    df_day.insert(len(df_day.columns),'Operator', 'ToDo')
    df_day = df_day.drop_duplicates('Link').reset_index(drop=True)
    df_day['Link'] = df_day['Link'].str.lower()
    df_day['Link'] = df_day['Link'].str.replace("'", "", regex=True)
#     df_day.set_index('Link', inplace=True)
#     df_main.set_index('Link', inplace=True)
    match = 0
    added = 0
    amnt_do = df_day['Link'].count()
    #METHOD ONE
    print(f'Load DFs time =  {time.time() - get_time}')
#     display(df_day)
    method_one = time.time()
    for index, row in df_day.iterrows():
        if index % 598 == 0:
            print(f'Processed { round((index / amnt_do) * 100, 2)}%')
        if len(df_main[df_main['Link'] == row['Link']]) == 1: # checking if URL matches exisitng one
            index_main = df_main[df_main['Link'] == row['Link']].index
            df_main.at[index_main[0], 'Date update'] = row['Date input']
            df_main.at[index_main[0], 'Reviews'] = row['Reviews']
            df_main.at[index_main[0], 'Tytul'] = row['Tytul']
            match = match + 1
        else:
            df_main = pd.concat([df_main, row.to_frame().T])     
            added = added + 1

    print(f'Matched {match}; Added {added}; Time taken: {round((time.time() - method_one)/60,4)} minutes')
    df_main.drop_duplicates('Link', inplace=True)
    df_main.to_csv(path_main, index=False, encoding='utf-8')            
    return f'Matched {match}; Added {added}; Time taken: {round((time.time() - method_one)/60,4)} minutes'
    
##@@@@@@@@@@@@ FUNCTION TESTING BELOW
# sheet = 'AllLinks'
# ##VIATOR
# pp = r'G:\.shortcut-targets-by-id\1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2\MyOTAs\Pliki firmowe\Operators_Groups.xlsx'
# day = r'G:\.shortcut-targets-by-id\1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2\MyOTAs\Baza Excel\Viator\All Links\All Links Viator - 2023-03-08.xlsx'
# insert_new_links(pp, day, 'AllLinks')
# ##GYG
# pp = r'G:\.shortcut-targets-by-id\1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2\MyOTAs\Pliki firmowe\Operators_GYG - Copy.xlsx'
# day = r'G:\.shortcut-targets-by-id\1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2\MyOTAs\Baza Excel\Get Your Guide\AllLinks\All Links GYG - 2023-03-07.xlsx'
# insert_new_links(pp, day, sheet)
#@@@@@@@@@@@@ FUNCTION TESTING ABOVE



In [6]:
def get_operators_name_from_chrome(path_main_file):
#     print(f'--------------{path_main_file}')
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
#     path = path_gyg.replace('\\','/') # replaces backslashes with forward slashes
#     path = path_gyg[1:len(path_gyg)-1] # to remove quote marks
#     print(path)

    df = pd.read_csv(path_main_file, encoding='latin-1')    
    df['Link'] = df['Link'].str.lower()
    df.drop_duplicates(subset=['Link'], inplace=True)              
    df.reset_index()
    timeS=time.time()
    countDone = 0
    countFailed = 0
    start = time.time()
    if 'GYG' in path_main_file:
        for index, row in df.iterrows():
            notFound = False
            if (row["Operator"] == "ToDo" and row['Link'] != 'tytul url'):
#             or ('Â' in str(row["Operator"])  and row['Link'] != 'tytul url'):
            #   print(row['GYG Link'], row['Tytul'])
#                 print(str(row["Operator"]))
                url = row['Link']
                driver.get(url)
                try:
                    elem = driver.find_element(By.CLASS_NAME, "supplier-name__link")
#                     print(f"Re: {re.sub('[^A-Za-z0-9 ]+', '', elem.text)}")
                except:
                    notFound = True


                if notFound == True:
                    df.at[index,'Operator'] = 'Incorrect URL'
                    countDone = countDone + 1    
                    countFailed = countFailed + 1
                else:
                    df.at[index,'Operator'] = re.sub('[^A-Za-z0-9 ]+', '', elem.text)
                    countDone = countDone + 1

                
                print(f'Index: {index} | Total time: {time.time() - timeS} | Avg per record: {(time.time() - timeS) / countDone} | Total done | {countDone} | {((countDone - countFailed)/countDone)*100}%')

                if countDone % 100 == 0:
                    print('INSERTING DF TO EXCEL')
                    df.to_csv(path_main_file, index=False)

        
        df.to_csv(path_main_file, index=False)    

        driver.quit()
        return f'Done: {countDone} Accuracy: {((countDone - countFailed)/countDone)*100}%'
    else:
        driver.quit()
        main_getoperator()
#         df['Length'] = df['Operator'].str.len()
#         for index, row in df.iterrows():
#             notFound = False
#             # VERIFY IF IT'S GONNA WORK ON THURSDAY OR CHANGE THE ISWEEKDAY
#             if row["Operator"] == "ToDo" or (datetime.datetime.today().isoweekday() == 4 and row["Operator"] == "Error"): # or row['Length'] > 75:
#         #         print(row['GYG Link'], row['Tytul'])
#                 url = row['Link']
#                 driver.get(url)
#                 try:
#                     elem = driver.find_element(By.CLASS_NAME, "supplierName__1So1") #supplierName__1JZV
#                 except:
#                     notFound = True

# # IF WAS NOT FOUND IN BASE APPROACH PAGE REFRESH
#                 if notFound == True:
#                     time.sleep(1)
#                     driver.get(url)
#                     try:
#                         elem = driver.find_element(By.CLASS_NAME, "supplierName__1So1") #supplierName__1JZV
#                     except:
#                         notFound = False
#                     # CHECK OPEN SEE MORE
#                     if notFound == False:
# # try to open see more
#                         try:
#                             additional_info_section = driver.find_element(By.XPATH, "//div[@data-automation='additional-info-section']")
#                             see_more_button = additional_info_section.find_element(By.CLASS_NAME, 'seeMoreLink__2-eS')
#                             driver.execute_script("arguments[0].scrollIntoView();", see_more_button)
#                             see_more_button.click()
# #                             try:
# #                                 see_more_buttons = driver.find_all(By.CLASS_NAME,'seeMoreLink__2-eS')
# #                                 print(see_more_buttons)
# #                                 for buttons in see_more_buttons:
# #                                     print(buttons.text.strip())
# #                                     driver.execute_script("arguments[0].scrollIntoView();", buttons)
# #                                     buttons.click()
# #                                     try:
# #                                         elem = driver.find_element(By.CLASS_NAME, "supplierName__1So1") #supplierName__1JZV
# #                                         Found = 'YYYY'
# #                                         df.at[index,'Operator'] = elem.text
# #                                     except:
# #                                         pass
# #                                     driver.get(url)
# #                                     time.sleep(1)
# #                             except:
# #                                 print('No for luup')

#                         except:
#                             pass                       
#                         try:
#                             elem = driver.find_element(By.CLASS_NAME, "supplierName__1So1") #supplierName__1JZV
#                             Found = 'YYY'
#                             df.at[index,'Operator'] = re.sub('[^A-Za-z0-9 ]+', '', elem.text)
#                         except:
#                             notFound = True
#                             df.at[index,'Operator'] = 'Incorrect URL'
#                             Found = "N"
#                             countFailed = countFailed + 1
#                     else:
#                         if len(elem.text) > 75:
#                             df.at[index,'Operator'] = 'Incorrect_Operator'
#                             Found = "Long"
#                         else:
#                             df.at[index,'Operator'] = re.sub('[^A-Za-z0-9 ]+', '', elem.text)
#                             Found = "YY"
#                     countDone = countDone + 1  
#                 else:
#                     if len(re.sub('[^A-Za-z0-9 ]+', '', elem.text)) > 75:
#                         df.at[index,'Operator'] = 'Incorrect_Operator'
#                         Found = "Long"
#                     else:
#                         df.at[index,'Operator'] = re.sub('[^A-Za-z0-9 ]+', '', elem.text)
#                         Found = "Y"
#                     countDone = countDone + 1


#                 print(f'Index: {index} | Total time: {time.time() - timeS} | Avg per record: {(time.time() - timeS) / countDone} | Total done| {countDone} | {Found} - {((countDone - countFailed)/countDone)*100}%')

#                 if countDone % 100 == 0:
#                     print('INSERTING DF TO EXCEL')
#                     df.to_csv(path_main_file, index=False)
#         end = time.time()
#         df.drop(columns='Length', inplace=True)
#         df.to_csv(path_main_file, index=False)            
#         driver.quit()
#         return f'Done: {countDone} Accuracy: {((countDone - countFailed)/countDone)*100}% Time taken: {round((end - start)/60,4)} min'

## function testing
# get_operators_name_from_chrome(r'G:\.shortcut-targets-by-id\1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2\MyOTAs\Pliki firmowe\Operators_Groups.csv')

In [7]:
# TO FIX ANY DUPLCIATES IN OPERATOR UID 


# Function to extract UID from URL
def extract_uid_gyg(link):
    try:
        # Split the link by '-' and take the last part
        parts = link.split('-')
        return parts[-1].replace('/', '').lower()
    except Exception as e:
        return None
    
def extract_uid_viator(link):
    try:
        # Split the link by '/' and take the last part
        parts = link.split('/')
        return parts[-1].lower()
    except Exception as e:
        return None
    

# df = pd.read_excel(r'G:\.shortcut-targets-by-id\1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2\MyOTAs\Pliki firmowe\Operators_GYG.xlsx')
# df_raw = pd.read_excel(r'G:\.shortcut-targets-by-id\1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2\MyOTAs\Pliki firmowe\Operators_GYG.xlsx')
def clean_add_uid(site, df):
    df = df[df['Link'].str.strip().str.len() > 1]
    # Convert the 'Link' column to string
    # Apply the function to create a new column 'UID'
    if site == 'GYG':
        df['uid'] = df['Link'].apply(extract_uid_gyg)
    elif site == 'Viator':
        df['uid'] = df['Link'].apply(extract_uid_viator)
    # Convert 'Date' column to datetime format
    df['Date input'] = pd.to_datetime(df['Date input'])
    # Sort by 'UID' and 'Date' to get the latest entry for each UID
    df = df.sort_values(by=['uid', 'Date input'], ascending=[True, False])
    # Drop duplicates to keep only the latest URL for each UID
    df = df.drop_duplicates(subset='uid')
    # Reset index for clarity
    df = df.reset_index(drop=True)
    return df



# df.to_csv(r'G:\.shortcut-targets-by-id\1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2\MyOTAs\Pliki firmowe\Operators_GYG.csv',index=False)
# df_viator.to_csv(r'G:\.shortcut-targets-by-id\1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2\MyOTAs\Pliki firmowe\Operators_Groups.csv',index=False)




In [8]:
def insert_data(data, cursor, insert_query):
    midpoint = int(round(len(data) / 2,0))
    print(midpoint)
    if len(data) == 20:  # Base case: only one row left.
        try:
            cursor.execute(insert_query, data[0])
        except pyodbc.Error as e:
            print(f"Error encountered with data {data[0]}: {e}")
        return
    
    first_half = data[:midpoint]
    print(first_half)
    second_half = data[midpoint:]
    print(second_half)

    try:
        cursor.executemany(insert_query, first_half)
    except pyodbc.Error:
        insert_data(first_half, cursor, insert_query)  # Recursively handle the problematic half

    try:
        cursor.executemany(insert_query, second_half)
    except pyodbc.Error:
        insert_data(second_half, cursor, insert_query)  # Recursively handle the problematic half
        

In [9]:
def find_problematic_bytes(file_path, encoding='windows-1252'):
    with open(file_path, 'rb') as file:
        byte_position = 0
        try:
            # Read the file byte by byte
            while byte := file.read(1):
                # Attempt to decode the byte(s)
                byte.decode(encoding)
                byte_position += 1
        except UnicodeDecodeError as e:
            # Log the position and the byte that caused the error
            print(f"Error at byte position {byte_position}: {e}")
            print(f"Problematic byte sequence: {byte}")
            
            # Optionally, read a few more bytes to get more context
            context = file.read(500)
            print(f"Following bytes: {context}")
            
            # Handle the error (skip, replace, etc.) or exit
            # ...

# Usage
# find_problematic_bytes(r'G:\.shortcut-targets-by-id\1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2\MyOTAs\Pliki firmowe\Operators_GYG.csv')


In [10]:
def upsert_df_to_sql_db(path_df_main, database_name):
    import pandas as pd
    import pyodbc
    import time
    from io import BytesIO

    df_main = pd.read_excel(path_df_main, sheet_name='AllLinks', engine='openpyxl')
    df_main['Reviews'] = df_main['Reviews'].fillna(0)
    df_main['Operator'] = df_main['Operator'].fillna('Error')
    df_main['Tytul'] = df_main['Tytul'].fillna('Error')
    df_main = df_main[df_main['City'].str.len() >= 3]

    if 'GYG' in path_df_main:
        table_name = 'Operators_GYG'
        df_main = clean_add_uid('GYG', df_main)
    else:
        table_name = 'Operators_Viator'
        df_main = clean_add_uid('Viator', df_main)
    df_main = df_main.drop_duplicates(subset=['uid'])

    server = 'sqlserver-myotas.database.windows.net'
    database = database_name
    username = 'azureadmin'
    password = 'brudnyHarry!66'
    driver = '{ODBC Driver 18 for SQL Server}'

    try:
        cnxn = pyodbc.connect(f'DRIVER={driver};SERVER=tcp:{server};PORT=1433;DATABASE={database};UID={username};PWD={password}')
        print('Connected')
    except:
        return "Couldn't connect to database"

    cursor = cnxn.cursor()
    cursor.fast_executemany = True

    # Create table if it doesn't exist
    create_table_query = f"""
        IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='{table_name}' AND xtype='U')
        CREATE TABLE [dbo].[{table_name}] (
            [Tytul]       NVARCHAR (MAX) NULL,
            [Link]        NVARCHAR (MAX) NULL,
            [City]        NVARCHAR (255) NULL,
            [Operator]    NVARCHAR (MAX) NULL,
            [Reviews]     NVARCHAR (255) NULL,
            [Date input]  NVARCHAR (255) NULL,
            [Date update] NVARCHAR (255) NULL,
            [uid]         NVARCHAR (255) NOT NULL PRIMARY KEY
        );
    """
    print('Ensuring table exists...')
    cursor.execute(create_table_query)
    cnxn.commit()

    # Upsert query
    merge_query = f"""
        MERGE [dbo].[{table_name}] AS target
        USING (VALUES (?, ?, ?, ?, ?, ?, ?, ?)) AS source (([Tytul], [Link], [City], [Operator], [Date input], [Date update], [uid], [Reviews]))
        ON target.[uid] = source.[uid]
        WHEN MATCHED THEN
            UPDATE SET
                target.[Tytul] = source.[Tytul],
                target.[Link] = source.[Link],
                target.[City] = source.[City],
                target.[Operator] = source.[Operator],
                target.[Date input] = source.[Date input],
                target.[Date update] = source.[Date update],
                target.[Reviews] = source.[Reviews]
        WHEN NOT MATCHED THEN
            INSERT ([Tytul], [Link], [City], [Operator], [Date input], [Date update], [uid], [Reviews])
            VALUES (source.[Tytul], source.[Link], source.[City], source.[Operator], source.[Date input], source.[Date update], source.[uid], source.[Reviews]);
    """
    data_list = [tuple(row) for row in df_main.values]
    print('Upserting data...')
    try:
        cursor.executemany(merge_query, data_list)
        cnxn.commit()
        print(f'Successfully upserted: {len(data_list)} rows')
    except pyodbc.DataError as e:
        print(e)

    cnxn.close()
    return f'Successfully upserted: {len(data_list)} rows to {table_name} table'


In [12]:
def insert_df_to_sql_db(path_df_main, database_name):
    # Open the file in binary mode and read the content
    # with open(path_df_main, 'rb') as file:
    #     binary_content = file.read()

    # # Use pandas to read the binary content directly
    # # We use the 'io.BytesIO' to create an in-memory binary stream
    # # Create an in-memory binary stream
    # data_stream = io.BytesIO(binary_content)

    # # Read the stream into a DataFrame using the appropriate engine
    df_main = pd.read_excel(path_df_main, sheet_name=0, engine='openpyxl')
    df_main['Reviews'] = df_main['Reviews'].fillna(0)
    df_main['Operator'] = df_main['Operator'].fillna('Error')
    df_main['Tytul'] = df_main['Tytul'].fillna('Error')
    df_main = df_main[df_main['City'].str.len() >= 3]

    


#     df_main['Date input'] = pd.to_datetime(df_main['Date input'])
#     df_main['Date update'] = pd.to_datetime(df_main['Date update'])
    # Define the table name
    if 'GYG' in path_df_main:
        table_name = 'Operators_GYG'
        #### CLEAN FROM UID AND DUPLCIATED URL BASED ON UID
        df_main = clean_add_uid('GYG', df_main)
    else:
        table_name = 'Operators_Viator'
        #### CLEAN FROM UID AND DUPLCIATED URL BASED ON UID
        df_main = clean_add_uid('Viator', df_main)
    df_main = df_main.drop_duplicates(subset=['uid'])
    server = 'sqlserver-myotas.database.windows.net'
    database = database_name
    username = 'azureadmin'
    password = paaas  
    driver = '{ODBC Driver 18 for SQL Server}'
    try:
        cnxn = pyodbc.connect('DRIVER='+driver+';SERVER=tcp:'+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
        print('Connected')
    except:
        return "Coundn't connect to database"

    # Read data into pandas dataframe
    data = df_main

    # Create a cursor and set fast_executemany to True
    cursor = cnxn.cursor()
    cursor.fast_executemany = True
    # Drop the table if it exists
    start_drop_table = time.time()
    print('Droping table...')
    cursor.execute(f"IF OBJECT_ID('{table_name}', 'U') IS NOT NULL DROP TABLE {table_name}")
    end_drop_table = time.time()
    # Create the table
    start_create_table = time.time()
    create_table_query = f"""
    CREATE TABLE [dbo].[{table_name}] (
        [Tytul]       NVARCHAR (MAX) NULL,
        [Link]        NVARCHAR (MAX) NULL,
        [City]        NVARCHAR (255) NULL,
        [Operator]    NVARCHAR (MAX) NULL,
        [Reviews]     NVARCHAR (255) NULL,
        [Date input]  NVARCHAR (255) NULL,
        [Date update] NVARCHAR (255) NULL,
        [uid]         NVARCHAR (255) NOT NULL,
        CONSTRAINT [PK_{table_name}] PRIMARY KEY CLUSTERED ([uid] ASC)
    );
    """


    # CONSTRAINT [PK_{table_name}] PRIMARY KEY CLUSTERED ([uid] ASC)
    print('Creating table...')
    cursor.execute(create_table_query)
    end_create_table = time.time()
    # Note start time
    start = time.time()

    # Use executemany to insert data into the table
    insert_query = f"INSERT INTO {table_name} ([Tytul], [Link], [City], [Operator], [Date input], [Date update], [uid], [Reviews])\
    VALUES (?, ?, ?, ?, ?, ?, ?, ?)"
    data_list = [tuple(row) for row in data.values]
    # print(data_list)
    print('Insert data...')
    try:
        cursor.executemany(insert_query, data_list)
        cnxn.commit()
        print(f'Sucessfully exectued inserted: {len(data_list)} rows')
    except pyodbc.DataError as e:
        ## Print the error message and the row causing the error
        print(e)
    # # insert_data(data_list, cursor, insert_query)
    # for i, row in enumerate(data_list):
    #     print(i,row)
    #     try:
    #         cursor.execute(insert_query, row)
    #     except pyodbc.DataError:
    #         print(f"Row {i}: {row}")
    #     cnxn.commit()

    # # Commit the changes and close the connection
    cnxn.close()

    # Calculate and print the execution time
    end = time.time()
    print(f"Drop time: {round(end_drop_table - start_drop_table, 4) } Create time: {round(end_create_table - start_create_table,4)} Execution time: {round(end - start,4)} seconds")
    return f'Sucesfully inserted: {len(data_list)} rows to {table_name} table in {round(end - start,4)} seconds'



In [13]:
# path_df_main = r'G:\.shortcut-targets-by-id\1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2\MyOTAs\Pliki firmowe\Operators_Groups.csv'
# # path_df_main = r'G:\.shortcut-targets-by-id\1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2\MyOTAs\Pliki firmowe\Operators_GYG.csv'
# df_main = pd.read_csv(path_df_main)

In [14]:
def upload_daily_to_sql(file_path):
    # Set up database connection details
    server = 'sqlserver-myotas.database.windows.net'
    database = 'OTAs'
    username = 'azureadmin'
    password = 'brudnyHarry!66'   
    driver = '{ODBC Driver 18 for SQL Server}'
    print(pyodbc.drivers())
    drive_path = r'G:\.shortcut-targets-by-id\1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2\MyOTAs'
    viator_daily_path = rf'{drive_path}\Baza Excel\Viator\Daily\\'
    viator_report_path = rf'{drive_path}\Baza Excel\Viator\Daily\ImportReports\\'
    gyg_report_path = rf'{drive_path}\Baza Excel\Get Your Guide\ImportReports\\'
    gyg_daily_path = rf'{drive_path}\Baza Excel\Get Your Guide\\'
    try:
        cnxn = pyodbc.connect('DRIVER='+driver+';SERVER=tcp:'+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password, timeout=300)
    except:
        return "Coundn't connect to database - retry"

    # Get path to Excel file from user input
#     excel_path = input("Enter path to Excel file: ")
    if '||' in file_path:
        files_upload = file_path.split('||')
    else:
        print('Notsplited')
        files_upload = file_path
    
    for file_upload in files_upload:
        excel_path = file_upload
        # excel_path = r'G:\.shortcut-targets-by-id\1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2\MyOTAs\Baza Excel\Viator\Daily\TestVPN_Viator - 2023-05-10.xlsx'
        # List of sheet names to exclude
        exclude_sheets = ['Sheet1', 'Data', 'Re-Run', 'DONE']
        date_of_import = excel_path.split()[-1].split('.')[0]

        # Save report to file ImportOfVPN_Viator - 2023-05-10.txt
        if 'Viator' in excel_path:
            report_path = f"{viator_report_path}ImportOfVPN_Viator - {date_of_import}.txt"
            folder_path = viator_report_path
            excel_path = f"{viator_daily_path}{file_upload}"
            header = ['Tytul', 'Tytul Url', 'Cena', 'Opinia','IloscOpini', 'Przecena', 'Tekst', 'Data zestawienia', 'Pozycja', 'Kategoria', 'SiteUse', 'Miasto']
#             header = ['Tytul', 'Tytul Url', 'Cena', 'IloscOpini', 'Opinia', 'RozmiarCena', 'Data zestawienia', 'Pozycja', 'Kategoria', 'SiteUse', 'Miasto']
        elif 'GYG' in excel_path:
            report_path = f"{gyg_report_path}ImportOfGYG - {date_of_import}.txt"
            folder_path = gyg_report_path
            excel_path = f"{gyg_daily_path}{file_upload}"
#             header = ['Tytul', 'Tytul URL', 'Cena', 'Opinia', 'IloscOpini', 'Przecena', 'Tekst', 'Data zestawienia', 'Pozycja', 'Kategoria', 'VPN_City', 'Booked', 'SiteUse', 'Miasto']
            header = ['Tytul', 'Tytul URL', 'Cena', 'Opinia', 'IloscOpini', 'Przecena', 'Tekst', 'Data zestawienia', 'Pozycja', 'Kategoria', 'Booked', 'SiteUse', 'Miasto', 'VPN_City']

        files = os.listdir(folder_path)

        if any(date_of_import in file for file in files):
            print(f'Import report already exisit for file {file_path}')
            print(f'Import report already exisit for file {file_upload}')
    #         continue
        else:
            # Load Excel file into pandas dataframe
            xls = pd.ExcelFile(excel_path)

            # Initialize report string
            report_str = ""
            cursor = cnxn.cursor()
            cursor.fast_executemany = True
            i = 1
            # Iterate over each sheet in the Excel file
            for sheet_name in xls.sheet_names:
                # Skip excluded sheets
                if sheet_name in exclude_sheets:
                    print('next', sheet_name)
                    continue
                print(f'{i} - {sheet_name}')
                start = time.time()
                df = pd.read_excel(excel_path, sheet_name=sheet_name, header=None)

                print(f'DF read {round(time.time() - start, 4)}s')
                df.columns = header
                df['Data zestawienia'] = df['Data zestawienia'].astype('str')
                df['IloscOpini'] = df['IloscOpini'].fillna(0)
                df['Opinia'] = df['Opinia'].fillna('N/A')
                df = df[df['Tytul'] != 'Tytul']
                df = df[df['Data zestawienia'] != 'Data zestawienia']
                df = df[df['Data zestawienia'].str.len() > 4]
#                 display(df)
                if sheet_name == 'Mt-Vesuvius':
                    sheet_name = 'Mount-Vesuvius'
                    df['Miasto'] = 'Mount-Vesuvius'
                # Insert Dataframe into SQL Server:
                if 'Viator' in excel_path:
                    insert_query = f"INSERT INTO [{sheet_name}] ([Tytul], [Tytul Url], [Cena], [Opinia], [IloscOpini],\
                    [Data zestawienia], [Pozycja], [Kategoria], [SiteUse], [Miasto])\
                    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"
                    df['Cena'] = df['Cena'].map(lambda x: x.split(x[0])[1].strip() if not x[0].isnumeric() else x)
                    df = df.drop(columns=('Przecena'))
                    df = df.drop(columns=('Tekst'))
                    
                    
                elif 'GYG' in excel_path:
#                     insert_query = f"INSERT INTO [{sheet_name}] ([Tytul], [Tytul Url], [Cena], [Opinia], [IloscOpini], [Przecena],\
#                     [Tekst], [Data zestawienia], [Pozycja], [Kategoria], [VPN_City], [Booked], [SiteUse], [Miasto])\
#                     VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"
                    insert_query = f"INSERT INTO [{sheet_name}] ([Tytul], [Tytul Url], [Cena], [Opinia], [IloscOpini], [Przecena],\
                    [Data zestawienia], [Pozycja], [Kategoria], [Booked], [SiteUse], [Miasto], [VPN_City])\
                    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"
##                    USED WHEN IMPORT GYG FROM PYTHON
                    df = df.drop(columns=('Tekst'))
                    df['Booked'] = df['Booked'].astype('str')
                    df['Przecena'] = df['Przecena'].astype('str')
                    df['Cena'] = df['Cena'].map(lambda x: x.split(x[0])[1].strip() if not x[0].isnumeric() else x)
                    df['Booked'] = df['Booked'].map(lambda x: x.split('Booked')[1].split()[0] if len(x) > 5 else x)
                    df['Przecena'] = df['Przecena'].map(lambda x: x.split()[1].replace(",", "") if len(x) > 4 else x)
##                  _________________
                    df['Przecena'] = df['Przecena'].fillna("NULL")
                    df['VPN_City'] = df['VPN_City'].fillna("NULL")
                    df['Booked'] = df['Booked'].fillna("NULL")

                data_list = [tuple(row) for row in df.values]
#                 print(data_list)


            # FOR TESTING PURPOSE IN CASE OF ANY ERROR
#                 for i, row in enumerate(data_list):
#                     print(i, row)
#                     try:
#                         cursor.execute(insert_query, row)
#                     except pyodbc.DataError:
#                         print(f"Row {i}: {row}")
#                     cnxn.commit()
            ##############################

                start_1 = time.time()
                try:
                    cursor.executemany(insert_query, data_list)
                    cnxn.commit()
            #         print(f'Sucessfully exectued inserted: {len(data_list)} rows')
                except pyodbc.DataError as e:
                    # Print the error message and the row causing the error
                    print(e)
                    print(e.with_traceback())
                print(f'DF insert {round(time.time() - start_1, 4)}s')
                report_str += f"\n{i} - {sheet_name} \n Import successful for sheet: {sheet_name}\n Sucessfully exectued inserted: {len(data_list)} rows \n"
                i = i +1

            #     except:
            #         report_str += f"Import failed for sheet: {sheet_name}\n"
            #         print(report_str)

            # Close database connection
            
            print(report_str)


            with open(report_path, "w") as f:
                f.write(report_str)

            print("Data upload complete!")
    cursor.close()
    cnxn.close()
    return "Done"

In [15]:
def start():

    sg.theme('DarkBlue2')  # please make your windows colorful

    layout = [
        [
            sg.Image(filename='logo_color.png', key='image')
        ], 
#               [sg.Text('_'  * 30)], 
#               [sg.FileBrowse('Browse File', file_types=(("Excel File", "*.xlsx"),), size=(36, 1), key="File"),
#               [sg.Text('_'  * 60)], 
#                sg.InputText('asdasd', size =(75, 2), key='PathExcel'),
#                ],
        
         [
            sg.Button('GYG file (Operators)', size=(20,1)),    
            sg.Button('Viator file (Operators)', size=(20,1)),
        ],
        
        [
            sg.Text('Path main: '),
            sg.Input('', size= (75,1), tooltip='Main excel file', key='FilePath')
#             sg.Text('Excel path GYG/Viator: '),
#             sg.InputText('', size= (30,2), tooltip='Excel file'),
#             sg.Button('Get Operators Name'),
#             sg.Button('Insert Data to SQL')
        ],
        [
              
            sg.Button("Add Today GYG (AllLinks)", size=(20,1)),
            sg.Button("Add Today Viator (AllLinks)", size=(20,1))
        ],
        [
            sg.Text('All Links Path: '),
            sg.Input('', size= (70,1), tooltip='All Links excel file', key='Date_Insert'),            
        ],
        [
            sg.Button("Insert New Links"),
            sg.Button('Get Operators Name'),
            sg.Button('Insert Data to SQL')
            
            
        ],
        [
            [sg.Text('_' * 90, pad=(0, 10))],
        ],
        [
              
            sg.Button("Add Today GYG (Daily)", size=(20,1)),
            sg.Button("Add Today Viator (Daily)", size=(20,1)),
            sg.Button("Add Today files (Daily)", size=(20,1)),
            sg.Button("Collect: All Links Viator", size=(20,1))
            
        ],
        [
            sg.Text('Daily Path: '),
            sg.Input('', size= (75,1), tooltip='Daily excel file', key='Daily_file'),            
        ],
        [
            sg.Button("Upload daily file to DB"),           
            
        ],
        [
            sg.Text('---', size=(70,1), key='Done'),
            sg.Button('Exit')
        ]
        
        ]

    window = sg.Window('Get Data MyOTAs', layout, default_element_size=(50, 1), grab_anywhere=False, keep_on_top=False)
    
    
    while True:  # Event Loop
        filePath = ''
        date_add = ''
        event, values = window.Read()
        if event == 'Insert New Links':
#             print(f'VALUES IN FUNCTION WILL BE{values['FilePath']},{values['Date_Insert']}')
            window['Insert New Links'].Update(button_color=('black','yellow')) 
            window['Done'].update('Working...')
            window.Refresh()
#             temp_val = 
#                 day_file_path = fr'G:\.shortcut-targets-by-id\1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2\MyOTAs\Baza Excel\Get Your Guide\AllLinks\All Links GYG - {temp_val}.xlsx'
#             day_file_path = temp_val
            print(values['FilePath'],values['Date_Insert'],'AllLinks')
            text = insert_new_links(values['FilePath'],values['Date_Insert'],'AllLinks')
#             current_time = datetime.datetime.now()             
            window['Insert New Links'].Update(button_color=('white','black')) 
            window['Done'].update(f'{text}')
            window.Refresh()
        if event == 'GYG file (Operators)':
            window['GYG file (Operators)'].Update(button_color=('black','yellow')) 
            window['Viator file (Operators)'].Update(button_color=('black','white')) 
            #Reset color of buttons
            window['Insert New Links'].Update(button_color=('black','white')) 
            window['Get Operators Name'].Update(button_color=('black','white'))
            window['Insert Data to SQL'].Update(button_color=('black','white')) 
            filePath = r'G:\.shortcut-targets-by-id\1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2\MyOTAs\Pliki firmowe\Operators_GYG.xlsx'
            window['FilePath'].update(filePath)
            window.Refresh()
        if event == 'Viator file (Operators)':
            window['GYG file (Operators)'].Update(button_color=('black','white')) 
            window['Viator file (Operators)'].Update(button_color=('black','yellow')) 
            #Reset color of buttons
            window['Insert New Links'].Update(button_color=('black','white')) 
            window['Get Operators Name'].Update(button_color=('black','white'))
            window['Insert Data to SQL'].Update(button_color=('black','white')) 
            filePath = r'G:\.shortcut-targets-by-id\1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2\MyOTAs\Pliki firmowe\Operators_Groups.xlsx'
            window['FilePath'].update(filePath)
            window.Refresh()
        if event == 'Get Operators Name':
            #DEBUG COMMENT - ask if VPN is on to run script
            answer = sg.popup_yes_no('Do you have VPN on?')
            if answer == 'No':
                break
            
            if len(values['FilePath']) < 5:
                    pass
                    print('Empty')
            else:
                print(values['FilePath'])
                window['Done'].update('Working...')
                window.Refresh()
                text = get_operators_name_from_chrome(values['FilePath'])
                window['Get Operators Name'].Update(button_color=('white','black')) 
                window['Done'].update(f'{text}')
                window.Refresh()
        if event == 'Add Today GYG (AllLinks)':
            date_add = datetime.date.today()
            path = fr'G:\.shortcut-targets-by-id\1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2\MyOTAs\Baza Excel\Get Your Guide\All Links\All Links GYG - {date_add}.xlsx'
            window['Date_Insert'].update(path)
            window['Add Today GYG (AllLinks)'].Update(button_color=('black','yellow')) 
            window['Add Today Viator (AllLinks)'].Update(button_color=('black','white')) 
            window.Refresh()
#             print(values['FilePath'],values['Date_Insert'])
        if event == 'Add Today Viator (AllLinks)':
            date_add = datetime.date.today()
            path = fr'G:\.shortcut-targets-by-id\1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2\MyOTAs\Baza Excel\Viator\All Links\All Links Viator - {date_add}.xlsx'
            window['Date_Insert'].update(path)
            window['Add Today Viator (AllLinks)'].Update(button_color=('black','yellow')) 
            window['Add Today GYG (AllLinks)'].Update(button_color=('black','white')) 
            window.Refresh()
#             print(values['FilePath'],values['Date_Insert'])

        if event == 'Insert Data to SQL':
            print(values['FilePath'])
            
            window['Done'].update('Working...')
            window.Refresh()
            text = upsert_df_to_sql_db(values['FilePath'], 'OTAs')
            window['Done'].update(f'{text}')
            text_2 = upsert_df_to_sql_db(values['FilePath'], 'db_ota_future_price')
            text = "OTAs:" + text + "\n" + "db_ota_future_prcie" + text_2
            window['Done'].update(f'{text}')
            window['Insert Data to SQL'].Update(button_color=('white','black')) 
            window.Refresh()
            
        if event == 'Add Today GYG (Daily)':
            date_add = datetime.date.today()
            path = fr'GYG - {date_add}.xlsx'
            window['Daily_file'].update(path)
            window['Add Today GYG (Daily)'].Update(button_color=('black','yellow')) 
            window['Add Today Viator (Daily)'].Update(button_color=('black','white')) 
            window.Refresh()
#             print(values['FilePath'],values['Date_Insert'])
        if event == 'Add Today Viator (Daily)':
            date_add = datetime.date.today()
            path = fr'Viator - {date_add}.xlsx'
            window['Daily_file'].update(path)
            window['Add Today Viator (Daily)'].Update(button_color=('black','yellow')) 
            window['Add Today GYG (Daily)'].Update(button_color=('black','white')) 
            window.Refresh()
        if event == 'Add Today files (Daily)':
            date_add = datetime.date.today()
            path = fr'Viator - {date_add}.xlsx||GYG - {date_add}.xlsx'
            window['Daily_file'].update(path)
            window['Add Today files (Daily)'].Update(button_color=('black','yellow')) 
            window['Add Today Viator (Daily)'].Update(button_color=('black','white'))
            window['Add Today GYG (Daily)'].Update(button_color=('black','white')) 
            
            window.Refresh()
                     
        if event == 'Upload daily file to DB':
            print(values['Daily_file'])
            window['Done'].update('Working...')
            window.Refresh()
            text = upload_daily_to_sql(values['Daily_file'])
            window['Done'].update(f'{text}')
            window['Upload daily file to DB'].Update(button_color=('white','black')) 
            window.Refresh()
        if event ==  'Exit' or event == sg.WIN_CLOSED:      
            break    
        if event == "Collect: All Links Viator":
            window['Done'].update('Working...')
            window.Refresh()
            text = main_alllinks()
            window['Done'].update(f'{text}')
            window.Refresh()

    window.close()
start()

G:\.shortcut-targets-by-id\1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2\MyOTAs\Pliki firmowe\Operators_Groups.xlsx
Connected
Droping table...
Creating table...
Insert data...
Sucessfully exectued inserted: 113291 rows
Drop time: 0.0396 Create time: 0.039 Execution time: 520.2754 seconds
Connected
Droping table...
Creating table...
Insert data...
Sucessfully exectued inserted: 113291 rows
Drop time: 0.0367 Create time: 0.0398 Execution time: 498.6801 seconds


In [16]:

# df_main = pd.read_csv(r'G:\.shortcut-targets-by-id\1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2\MyOTAs\Pliki firmowe\Operators_Groups.csv')
# display(df_main)
# #Based on input read files

# df_day = pd.read_excel(fr'G:\.shortcut-targets-by-id\1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2\MyOTAs\Baza Excel\Viator\All Links\All Links Viator - 2023-08-22.xlsx', 'AllLinks')

# #SHORTEN DFS
# df_day = df_day[df_day['Miasto'] == 'Amsterdam'] #short version for testing
# display(df_day)
# # df_main = df_main.head(5000) #short version for testing
# #____

# df_day = df_day[['Tytul', 'Tytul URL', 'IloscOpini' ,'Data zestawienia', 'Miasto']]    
# df_day.rename(columns={'Tytul URL': 'Link', 'IloscOpini': 'Reviews', 'Miasto': 'City', 'Data zestawienia': 'Date input'}, inplace=True)
# df_day.insert(len(df_day.columns), 'Date update', df_day['Date input'])
# df_day.insert(len(df_day.columns),'Operator', 'ToDo')
# df_day.drop_duplicates('Link', inplace=True)
# df_day['Link'] = df_day['Link'].str.lower()
# df_day['Link'] = df_day['Link'].str.replace("'", "", regex=True)
# match = 0
# added = 0
# amnt_do = df_day['Link'].count()
# #METHOD ONE
# for index, row in df_day.iterrows():
#     if index % 598 == 0:
#         print(f'Processed { round((index / amnt_do) * 100, 2)}%')
#     if len(df_main[df_main['Link'] == row['Link']]) == 1: # checking if URL matches exisitng one
#         index_main = df_main[df_main['Link'] == row['Link']].index
#         df_main.at[index_main[0], 'Date update'] = row['Date input']
#         df_main.at[index_main[0], 'Reviews'] = row['Reviews']
#         df_main.at[index_main[0], 'Tytul'] = row['Tytul']
#         match = match + 1
#     else:
#         df_main = pd.concat([df_main, row.to_frame().T])     
#         added = added + 1
#         print('Adding...')
#         print(df_day[df_day['Link'] == row['Link']]['Reviews'] == df_main[df_main['Link'] == row['Link']]['Reviews'])
        